<img src="images/ProjectPythia_Logo_Final-01-Blue.svg" width=250 alt="Project Pythia Logo"> 

# Creating a Basic Interactive Dashboard with `hvPlot`

## Overview

ERA-5 Dataset is available from NCAR RDA in netcdf format. A subset of this dataset is processed into Zarr format and available from NCAR RDA endpoints. To learn how you can create Zarr files from NCAR RDA netcdf files, please see [this notebook](./05_data_preprocessing.ipynb).


By the end of this notebook, you should be able to:
* Understand the importance for interactive plots and the challenges associated with them
* Use `hvPlot` to generate basic interactive plots with `Xarray`

## Prerequisites


| Concepts | Importance | Notes |
| --- | --- | --- |
| [Intro to Xarray](https://foundations.projectpythia.org/core/xarray.html) | Necessary | |

- **Time to learn**: 30 minutes

## Imports

In [ ]:
import xarray as xr
import holoviews as hv
from holoviews import opts

hv.extension("bokeh")


## Data

As we mentioned above a subset of NCAR RDA data is available in Zarr format. 

In [ ]:
rda_url = 'https://data.rda.ucar.edu/'
annual_means = rda_url + 'pythia_era5_24/annual_means/'
xrds = xr.open_dataset(annual_means + "temp_2m_annual_1940_2023.zarr", engine= 'zarr')
xrds.load()


## Considerations for Interactive Plots

Add some markdown text on some of the following ideas:
* What are some reasons we want to make data visualuzation interactive?

## Baisc Interactivity using `hvPlot`

The `hvPlot` package is a familiar and high level API for data exploration and visualuzation.



<div style="text-align: left;">
    <img src="https://hvplot.holoviz.org/_images/diagram.svg" width="900">
</div>


One of the most powerfull features of `hvPlot` is that it provides an alternative plotting API that directly attaches to existing Python objects through the `.hvplot()` attribute. For the case of `Xarray`, importing `hvplot.xarray` adds a brand new set of plotting routines accessible either through `xr.DataArray.hvplot()` or `xr.Dataset.hvplot()`

In [ ]:
import hvplot.xarray

Before using `hvPlot`, let's take a look at the default `Xarray` plotting methods.

In [ ]:
xrds['VAR_2T'].plot()

We can replace the `.plot()` function call with `.hvplot()`. By default, `hvPlot` uses the `Bokeh` backend, which has naitive interactive tools, such as :
* Panning
* Box Select
* Scroll Zoom
* Saving
* Resetting

In [ ]:
xrds['VAR_2T'].hvplot()

If we wanted to plot ...

In [ ]:
xrds['VAR_2T'].isel(time=0).plot()

Switching

In [ ]:
xrds['VAR_2T'].isel(time=0).hvplot()

### Time Widget

Climate data typically comes with multiple timesteps. We can create a basic widget that allows us to seek through time by setting the `groupby='time'` parameter in our `.hvplot()` call. 

In [ ]:
xrds['VAR_2T'].hvplot(groupby='time', widget_location="bottom")

You may notice that our colorbar is dynamically changing as we change our time steps. We can fix the colorbar by setting a `clim` value, which is a tuple of the minimum and maximum desired colorbar range.

One suggestion is to use the minimum and maximum of the data variable you are visualuzing across time.

In [ ]:
clim = (xrds['VAR_2T'].values.min(), xrds['VAR_2T'].values.max())

In [ ]:
xrds['VAR_2T'].hvplot(clim=clim, groupby='time', widget_location="bottom")

You may have noticed that there is a slight lag when switching time steps. This is due to `hvPlot` plotting the full resolution of our dataset. We can instead rasterize the output by setting `rasterize=True`, which will significantly improve the perfromance of our interactive plot.

In [ ]:
xrds['VAR_2T'].hvplot(rasterize=True, clim=clim, groupby='time', widget_location="bottom")

### Animation Widget

Another usefull interactive feature is animations. Instead of manually scrolling through time, we can set up a widget that lets us animate our data across time. This can be achieved by adding a Scrubber widget to our plot by setting `widget_type="scrubber"`

In [ ]:
xrds['VAR_2T'].hvplot(
    rasterize=True,
    groupby="time",
    widget_type="scrubber",
    widget_location="bottom",
)

## Creating a Dashboard

### Dataset Widgets

In [ ]:
w_time = pn.widgets.IntSlider(name='Year', start=0, end=83)
w_var = pn.widgets.Select(name='Data Variable', options=list(xrds.data_vars))

dataset_controls = pn.WidgetBox(
                                '## Dataset Controls', 
                                w_var, 
                                )
dataset_controls

### Plotting Widgets

In [ ]:
w_cmap = pn.widgets.Select(name='Colormap', options=['inferno', 'plasma', 'coolwarm'])


w_plot_type = pn.widgets.Select(name='Plot Type', options=['Color Plot', 'Contour', 'Filled Contour'])


plot_controls = pn.WidgetBox(
                            '## Plot Controls',
                            w_plot_type,
                            w_cmap, 
                            )
plot_controls

### Animation Widgets

In [ ]:
w_player = pn.widgets.Player(
    value=0,
    start=0,
    end=83,
    name="Year",
    loop_policy="loop",
    interval=300,
    align="center",
    width_policy='fit'
)
w_player

### Plotting Function

In [ ]:
def plot_ds(time, var, cmap, plot_type):
    clim = (xrds[var].values.min(), xrds[var].values.max())
    
    if plot_type == "Color Plot":
        return xrds[var].isel(time=time).hvplot(cmap=cmap, 
                                                              title=str(f"{var} year {time}"),
                                                              clim=clim,
                                                              dynamic=False,
                                                              rasterize=True,
                                                              precompute=True,
                                               ).opts(framewise=False)
    
    elif plot_type == "Contour":
        return xrds[var].isel(time=time).hvplot.contour(cmap=cmap,
                                          dynamic=False,
                                          rasterize=True,
                                          title=str(f"{var} Year: {time}"),
                                          clim=clim,
                                          precompute=True,).opts(framewise=False)
    elif plot_type == "Filled Contour":
        return xrds[var].isel(time=time).hvplot.contourf(cmap=cmap,
                                          dynamic=False,
                                          rasterize=True,
                                          title=str(f"{var} Year: {time}"),
                                          clim=(200, 300),
                                          precompute=True,).opts(framewise=False)                 

### Putting it all Together

In [ ]:
controls = pn.Column(dataset_controls, plot_controls)

app = pn.Row(
    controls,
    pn.Column(pn.panel(
        hv.DynamicMap(pn.bind(
            plot_ds, 
            time=w_player,
            var=w_var,
            cmap=w_cmap,
            plot_type = w_plot_type
        )
                     )
    ),
             w_player)
)

app